---
## CSE 482 - Tweet Loading and Processing
### Jack Nugent
Load Tweets related to Elon Musk and prepare the text for sentiment analysis.

---

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import json
import pandas as pd
import re
import numpy as np

In [2]:
#!pip install emoji
import emoji

---
### Loading Tweets

In [3]:
data_json = []

In [4]:
files = ['output_geotag_day1.json', 'output_geotag_day2.json', 
         'output_geotag_day3.json', 'output_geotag_day4.json',
         'output_geotag_day5.json']

for file in files:
    f = open(file)

    for line in f:
        try:
            data_json.append(json.loads(line))
        # Discard Tweets with invalid chars
        except:
            pass

    f.close()

In [5]:
df = pd.DataFrame(columns=['Text', 'Location', 'Tweet_id', 'Date', 'Time'])

for tweet in data_json:
    date, time = tweet['data']['created_at'].split('T')
    # Handles Retweets that somehow slip through filter
    if 'RT @' not in tweet['data']['text']:
        df.loc[len(df.index)] = [tweet['data']['text'], tweet['includes']['places'][0]['full_name'], tweet['data']['id'], date, time[:-5]]

print("Number of Tweets:",len(df))
df

Number of Tweets: 2736


,Text,Location,Tweet_id,Date,Time
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,"Bay Harbor Islands, FL",1595482958764425218,2022-11-23,18:22:34
1,Twitter and Elon has the same vibes when Trump...,"New York, USA",1595483029048369152,2022-11-23,18:22:51
2,@RonFilipkowski Then why do I see him so frequ...,"West Virginia, USA",1595483099735154688,2022-11-23,18:23:08
3,@timothy_wall @elonmusk This is hysterical. Th...,"Port Orange, FL",1595483141879353344,2022-11-23,18:23:18
4,Elon Musk can show us the way but it’s we who ...,"Brooklyn, NY",1595483162289016832,2022-11-23,18:23:23
...,...,...,...,...,...
2731,@genzforchange @JDCocchiarella Especially not ...,"New Hampshire, USA",1596923713450283008,2022-11-27,17:47:37
2732,I hope elon buys IG next because bots/scammers...,"New Orleans, LA",1596923811605381120,2022-11-27,17:48:00
2733,@Liana53160 If its made in U S A,"Tuscaloosa, AL",1596923815036354560,2022-11-27,17:48:01
2734,@RookieMisteak @Knightengale Maine...Elon real...,"Fort Worth, TX",1596923839383904257,2022-11-27,17:48:07


---
### Processing Text


#### Cleaning Text

In [34]:
%%time
df['Words'] = df['Text'].str.split()
df['Hashtags'] = None
for x in range(len(df)):
    # Handle '@' Mentions
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('@').match(i)]
    
    # Handle links
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('http').match(i)]
    
    # Hashtags
    df['Hashtags'][x] = [word for word in df['Words'][x] if '#' in word]
    df['Words'][x] = [word for word in df['Words'][x] if word not in df['Hashtags'][x]]
    
    # Remove numbers
    df['Words'][x] = [word for word in df['Words'][x] if not word.isnumeric()]
    
    # Handle Emojis and Punctuation
    words = []
    for word in df.iloc[x]['Words']:
        for c in word:
            if c in emoji.EMOJI_DATA or c in '''!()-[]{};:'"\,<>./?@#$%^&*_~1234567890''':
                word = word.replace(c, '')
        # Lowercase Words
        words.append(word.lower())
    df['Words'][x] = words
    
    # Remove Empty Values
    for word in df['Words'][x]:
        if word == '':
            df['Words'][x].remove(word)
            

CPU times: user 3.26 s, sys: 78.5 ms, total: 3.34 s
Wall time: 3.22 s


---
### NLTK Cleaning

In [35]:
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package wordnet to /home/nugentj3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nugentj3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


---
#### Stopword Removal

In [36]:
stop_words = set(stopwords.words('english'))

In [37]:
# Stopword Removal
for x in range(len(df)):
    df['Words'][x] = [word for word in df['Words'][x] if word not in stop_words]
#df

---
#### Stemming

In [38]:
df['Stemmed'] = None
for x in range(len(df)):
    df['Stemmed'][x] = [SnowballStemmer(language='english').stem(word) for word in df['Words'][x]]
#df

---
#### Lemmatization

In [39]:
df['Lemmed'] = None
for x in range(len(df)):
    df['Lemmed'][x] = [WordNetLemmatizer().lemmatize(word) for word in df['Words'][x]]
#df

---
#### Part of Speech Tagging

In [40]:
#POS Tagging
df['POS_Tags'] = None
for x in range(len(df)):
    df['POS_Tags'][x] = nltk.pos_tag(df['Lemmed'][x])
#df

---
### Positive/Negative

In [41]:
pos_words = []
f = open('positive-words.txt')
for line in f:
    try:
        pos_words.append(line.replace('\n', ''))
    except:
        pass

f.close()
len(pos_words)

2006

In [42]:
neg_words = []
f = open('negative-words.txt', encoding='ISO-8859-1')
for line in f:
    try:
        neg_words.append(line.replace('\n', ''))
    except:
        pass

f.close()
len(neg_words)

4783

In [43]:
df['Count'] = None
df['Pos_count'] = None
df['Neg_count'] = None

for x in range(len(df)):
    df['Count'][x] = len(df['Lemmed'][x])
    df['Pos_count'][x] = len([word for word in df['Lemmed'][x] if word in pos_words])
    df['Neg_count'][x] = len([word for word in df['Lemmed'][x] if word in neg_words])
    

In [44]:
# Remove rows with no words.
df_fix = df.copy()
for idx, row in df[df['Count'] == 0].iterrows():
    df_fix = df_fix.drop(idx)

In [45]:
df_fix['Positive_ratio'] = df_fix['Pos_count']/df_fix['Count']
df_fix['Negative_ratio'] = df_fix['Neg_count']/df_fix['Count']

In [46]:
df_fix

,Text,Location,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,"Bay Harbor Islands, FL",1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[],"[econusyougovamerica, approv, disapprov, elon,...","[econusyougovamerica, approve, disapprove, elo...","[(econusyougovamerica, RB), (approve, VB), (di...",24,2,1,0.083333,0.041667
1,Twitter and Elon has the same vibes when Trump...,"New York, USA",1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[],"[twitter, elon, vibe, trump, got, elect, laugh...","[twitter, elon, vibe, trump, got, elected, lau...","[(twitter, NN), (elon, NN), (vibe, NN), (trump...",19,1,1,0.052632,0.052632
2,@RonFilipkowski Then why do I see him so frequ...,"West Virginia, USA",1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[],"[see, frequent, twitter, stop, post]","[see, frequently, twitter, stop, posting]","[(see, VB), (frequently, RB), (twitter, JJR), ...",5,0,0,0.0,0.0
3,@timothy_wall @elonmusk This is hysterical. Th...,"Port Orange, FL",1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[],"[hyster, thank, laugh, prep, thanksgiv, need]","[hysterical, thanks, laugh, prep, thanksgiving...","[(hysterical, JJ), (thanks, NNS), (laugh, IN),...",6,0,1,0.0,0.166667
4,Elon Musk can show us the way but it’s we who ...,"Brooklyn, NY",1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[],"[elon, musk, show, us, way, it, hard, work, ma...","[elon, musk, show, u, way, it’s, hard, work, m...","[(elon, NN), (musk, NN), (show, NN), (u, JJ), ...",10,1,1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,@genzforchange @JDCocchiarella Especially not ...,"New Hampshire, USA",1596923713450283008,2022-11-27,17:47:37,"[especially, believe, lies, democrats, propaga...",[],"[especi, believ, lie, democrat, propaganda, ma...","[especially, believe, lie, democrat, propagand...","[(especially, RB), (believe, VB), (lie, NN), (...",7,0,2,0.0,0.285714
2732,I hope elon buys IG next because bots/scammers...,"New Orleans, LA",1596923811605381120,2022-11-27,17:48:00,"[hope, elon, buys, ig, next, botsscammers, con...",[],"[hope, elon, buy, ig, next, botsscamm, control]","[hope, elon, buy, ig, next, botsscammers, cont...","[(hope, NN), (elon, NN), (buy, VB), (ig, JJ), ...",7,0,0,0.0,0.0
2733,@Liana53160 If its made in U S A,"Tuscaloosa, AL",1596923815036354560,2022-11-27,17:48:01,"[made, u]",[],"[made, u]","[made, u]","[(made, VBN), (u, NN)]",2,0,0,0.0,0.0
2734,@RookieMisteak @Knightengale Maine...Elon real...,"Fort Worth, TX",1596923839383904257,2022-11-27,17:48:07,"[maineelon, really, needs, get, edit, button, ...",[],"[maineelon, realli, need, get, edit, button, a...","[maineelon, really, need, get, edit, button, a...","[(maineelon, NN), (really, RB), (need, VB), (g...",20,1,2,0.05,0.1


---
#### Sort By Positive Ratio

In [47]:
df_fix.sort_values('Positive_ratio')[::-1][:50]

,Text,Location,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
2186,Respect!!💜 https://t.co/DjKy47TwRh,"Indiana, USA",1596892254253219840,2022-11-27,15:42:36,[respect],[],[respect],[respect],"[(respect, NN)]",1,1,0,1.0,0.0
156,@mazemoore Love it!! 😂😂😂,"Houston, TX",1595497183511515149,2022-11-23,19:19:06,[love],[],[love],[love],"[(love, NN)]",1,1,0,1.0,0.0
2243,"@MattWallace888 I love 420, but I prefer 422! ...","Los Angeles, CA",1596896403116417025,2022-11-27,15:59:06,"[love, prefer, love]","[#nikkiforpresident2024, #vibinwithnikki]","[love, prefer, love]","[love, prefer, love]","[(love, NN), (prefer, NN), (love, NN)]",3,3,0,1.0,0.0
2704,"@ProudElephantUS The best,100%","Surprise, AZ",1596922286707453953,2022-11-27,17:41:57,[best],[],[best],[best],"[(best, JJS)]",1,1,0,1.0,0.0
1113,@3rdtimewalter @MattWallace888 Good,"Ravenna, OH",1596183327152967682,2022-11-25,16:45:35,[good],[],[good],[good],"[(good, JJ)]",1,1,0,1.0,0.0
2698,@MattWallace888 Awesome,"Anchorage, AK",1596922063713079296,2022-11-27,17:41:04,[awesome],[],[awesom],[awesome],"[(awesome, NN)]",1,1,0,1.0,0.0
1727,@onthebus17 @chrislhayes Thank you!,"Pennsylvania, USA",1596556403292114944,2022-11-26,17:28:03,[thank],[],[thank],[thank],"[(thank, NN)]",1,1,0,1.0,0.0
184,@realTuckFrumper Good,"South Lebanon, OH",1595500171353522176,2022-11-23,19:30:58,[good],[],[good],[good],"[(good, JJ)]",1,1,0,1.0,0.0
557,@bennyjohnson Outstanding!,"Greater Northdale, FL",1595835514615386112,2022-11-24,17:43:30,[outstanding],[],[outstand],[outstanding],"[(outstanding, JJ)]",1,1,0,1.0,0.0
2501,@MattWallace888 Nice 😘👍,"San Diego, CA",1596911228626296832,2022-11-27,16:58:00,[nice],[],[nice],[nice],"[(nice, JJ)]",1,1,0,1.0,0.0


---
#### One/Two Word Responses

In [48]:
len(df_fix[df_fix['Count'] < 2])

197

In [49]:
df_fix[df_fix['Count'] > 2]

,Text,Location,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,"Bay Harbor Islands, FL",1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[],"[econusyougovamerica, approv, disapprov, elon,...","[econusyougovamerica, approve, disapprove, elo...","[(econusyougovamerica, RB), (approve, VB), (di...",24,2,1,0.083333,0.041667
1,Twitter and Elon has the same vibes when Trump...,"New York, USA",1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[],"[twitter, elon, vibe, trump, got, elect, laugh...","[twitter, elon, vibe, trump, got, elected, lau...","[(twitter, NN), (elon, NN), (vibe, NN), (trump...",19,1,1,0.052632,0.052632
2,@RonFilipkowski Then why do I see him so frequ...,"West Virginia, USA",1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[],"[see, frequent, twitter, stop, post]","[see, frequently, twitter, stop, posting]","[(see, VB), (frequently, RB), (twitter, JJR), ...",5,0,0,0.0,0.0
3,@timothy_wall @elonmusk This is hysterical. Th...,"Port Orange, FL",1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[],"[hyster, thank, laugh, prep, thanksgiv, need]","[hysterical, thanks, laugh, prep, thanksgiving...","[(hysterical, JJ), (thanks, NNS), (laugh, IN),...",6,0,1,0.0,0.166667
4,Elon Musk can show us the way but it’s we who ...,"Brooklyn, NY",1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[],"[elon, musk, show, us, way, it, hard, work, ma...","[elon, musk, show, u, way, it’s, hard, work, m...","[(elon, NN), (musk, NN), (show, NN), (u, JJ), ...",10,1,1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,@WalshFreedom @elonmusk Sell for a loss and wa...,"Port Townsend, WA",1596923698904457216,2022-11-27,17:47:33,"[sell, loss, walk, away]",[],"[sell, loss, walk, away]","[sell, loss, walk, away]","[(sell, VB), (loss, NN), (walk, NN), (away, RB)]",4,0,1,0.0,0.25
2731,@genzforchange @JDCocchiarella Especially not ...,"New Hampshire, USA",1596923713450283008,2022-11-27,17:47:37,"[especially, believe, lies, democrats, propaga...",[],"[especi, believ, lie, democrat, propaganda, ma...","[especially, believe, lie, democrat, propagand...","[(especially, RB), (believe, VB), (lie, NN), (...",7,0,2,0.0,0.285714
2732,I hope elon buys IG next because bots/scammers...,"New Orleans, LA",1596923811605381120,2022-11-27,17:48:00,"[hope, elon, buys, ig, next, botsscammers, con...",[],"[hope, elon, buy, ig, next, botsscamm, control]","[hope, elon, buy, ig, next, botsscammers, cont...","[(hope, NN), (elon, NN), (buy, VB), (ig, JJ), ...",7,0,0,0.0,0.0
2734,@RookieMisteak @Knightengale Maine...Elon real...,"Fort Worth, TX",1596923839383904257,2022-11-27,17:48:07,"[maineelon, really, needs, get, edit, button, ...",[],"[maineelon, realli, need, get, edit, button, a...","[maineelon, really, need, get, edit, button, a...","[(maineelon, NN), (really, RB), (need, VB), (g...",20,1,2,0.05,0.1


In [50]:
df_fix[df_fix['Count'] > 2].sort_values('Positive_ratio')[::-1][:50]

,Text,Location,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags,Count,Pos_count,Neg_count,Positive_ratio,Negative_ratio
2243,"@MattWallace888 I love 420, but I prefer 422! ...","Los Angeles, CA",1596896403116417025,2022-11-27,15:59:06,"[love, prefer, love]","[#nikkiforpresident2024, #vibinwithnikki]","[love, prefer, love]","[love, prefer, love]","[(love, NN), (prefer, NN), (love, NN)]",3,3,0,1.0,0.0
1356,@MichaelMcCros16 Interesting- I wonder what Tr...,"Manhattan, NY",1596535643697733632,2022-11-26,16:05:34,"[interesting, wonder, trump, thinking]",[],"[interest, wonder, trump, think]","[interesting, wonder, trump, thinking]","[(interesting, VBG), (wonder, NN), (trump, NN)...",4,3,0,0.75,0.0
1764,Doing great @elonmusk keep up the stellar work...,"West Virginia, USA",1596558626185183232,2022-11-26,17:36:53,"[great, keep, stellar, work]",[],"[great, keep, stellar, work]","[great, keep, stellar, work]","[(great, JJ), (keep, VB), (stellar, JJ), (work...",4,3,0,0.75,0.0
462,@RBReich Like this during trump's inauguration...,"Gibsonton, FL",1595825124225290247,2022-11-24,17:02:13,"[like, trumps, inauguration]",[],"[like, trump, inaugur]","[like, trump, inauguration]","[(like, IN), (trump, NN), (inauguration, NN)]",3,2,0,0.666667,0.0
608,@LauraFlowD @WholeMarsBlog I’m ready for some ...,"Baton Rouge, LA",1595842277305368579,2022-11-24,18:10:22,"[i’m, ready, goodness]",[],"[i'm, readi, good]","[i’m, ready, goodness]","[(i’m, JJ), (ready, JJ), (goodness, NN)]",3,2,0,0.666667,0.0
2722,"@CNN Elon is great, and the great.","Manhattan, NY",1596923466284171264,2022-11-27,17:46:38,"[elon, great, great]",[],"[elon, great, great]","[elon, great, great]","[(elon, RB), (great, JJ), (great, JJ)]",3,2,0,0.666667,0.0
2721,@voluntee01 Good morning Beautiful https://t.c...,"LaSalle, IL",1596923379361411072,2022-11-27,17:46:17,"[good, morning, beautiful]",[],"[good, morn, beauti]","[good, morning, beautiful]","[(good, JJ), (morning, NN), (beautiful, NN)]",3,2,0,0.666667,0.0
1809,@StephenKing Wow. Thank you sir. Civility! Lik...,"Connecticut, USA",1596561530128142337,2022-11-26,17:48:26,"[wow, thank, sir, civility, like, hot, cup, ch...",[],"[wow, thank, sir, civil, like, hot, cup, chocol]","[wow, thank, sir, civility, like, hot, cup, ch...","[(wow, NN), (thank, NN), (sir, JJ), (civility,...",8,5,0,0.625,0.0
253,@AdamParkhomenko Well this is a nice example o...,"Florida, USA",1595507064985554956,2022-11-23,19:58:22,"[well, nice, example, race, baiting, congratul...",[],"[well, nice, exampl, race, bait, congratul, am...","[well, nice, example, race, baiting, congratul...","[(well, RB), (nice, JJ), (example, NN), (race,...",10,6,0,0.6,0.0
1891,@kylegriffin1 It’s gained top brands that are ...,"Coral Gables, FL",1596566148233191424,2022-11-26,18:06:47,"[it’s, gained, top, brands, globalist, illumin...",[],"[it, gain, top, brand, globalist, illuminati, ...","[it’s, gained, top, brand, globalist, illumina...","[(it’s, NN), (gained, VBD), (top, JJ), (brand,...",8,4,0,0.5,0.0


In [51]:
df_fix.to_pickle('tweet_data.df')